# Streaming: Processing Unlimited Frames On-Disk

A key feature of trackpy is the ability to process an unlimited number of frames.

For feature-finding, this is straightforward: a frame is loaded, features are located, the locations are saved the disk, and the memory is cleared for the next frame. For linking, the problem is more challenging, but trackpy handles all this complexity for you, using as little memory as possible throughout.

When data sets become large, beginning-friendly file formats like CSV or Excel become impractical. We recommend using the [HDF5 file format](https://support.hdfgroup.org/HDF5/), which is trackpy can read and write out of the box. (HDF5 is [widely used](http://en.wikipedia.org/wiki/Hierarchical_Data_Format); you can be sure it will be around for many, many years to come.)

If you have some other format in mind, see the end of this tutorial, where we explain how to extend trackpy's interface to support other formats.

## PyTables

You need pytables, which is normally included with the Anaconda distribution. If you find that you don't have it, you can easily install it using conda. Type this command into a Terminal or Command Prompt.

    conda install pytables
    
## Locate Features, Streaming Results into an HDF5 File

In [1]:
import trackpy as tp
import pims

In [2]:
@pims.pipeline
def gray(image):
    return image[:, :, 0]

images = gray(pims.open('../sample_data/bulk_water/*.png'))
images = images[:10]  # We'll take just the first 10 frames for demo purposes.

In [3]:
# For this demo, we'll first remove the file if it already exists.
!rm -f data.h5

We can use `locate` inside a loop:

In [4]:
with tp.PandasHDFStore('data.h5') as s:  # This opens an HDF5 file. Data will be stored and retrieved by frame number.
    for image in images:
        features = tp.locate(image, 11, invert=True)  # Find the features in a given frame.
        s.put(features)  # Save the features to the file before continuing to the next frame.

or, equivalently, we can use `batch`, which accepts the storage file as `output`.

In [5]:
with tp.PandasHDFStore('data.h5') as s:
    tp.batch(images, 11, invert=True, output=s)

Frame 9: 563 features


We can get the data for a given frame:

In [6]:
with tp.PandasHDFStore('data.h5') as s:
    frame_2_results = s.get(2)
    
frame_2_results.head()  # Display the first few rows.

,y,x,mass,size,ecc,signal,raw_mass,ep,frame
0,5.644760,295.595642,249.545063,2.178398,0.182386,15.450905,8915.0,0.116651,2
1,6.359559,68.303687,227.102408,2.428999,0.082812,10.358149,9269.0,0.072415,2
2,6.438481,336.577209,209.061966,2.006389,0.113846,14.415090,8815.0,0.140979,2
3,6.902141,432.482819,346.652706,2.863077,0.485857,13.551911,9127.0,0.085406,2
4,8.217391,36.224806,256.105224,2.625996,0.144332,9.667605,9530.0,0.056592,2


Or dump all the data, if your machine has enough memory to hold it:

In [7]:
with tp.PandasHDFStore('data.h5') as s:
    all_results = s.dump()
    
all_results.head()  # Display the first few rows.

,y,x,mass,size,ecc,signal,raw_mass,ep,frame
0,4.949033,103.834828,187.762850,2.263025,0.076349,9.835619,8983.0,0.106633,0
1,5.249298,585.934868,157.812948,2.899399,0.054051,4.164631,9001.0,0.103610,0
2,5.789880,294.498262,229.409164,2.109955,0.175696,15.417997,8917.0,0.119408,0
3,5.853706,338.236622,180.496897,2.060928,0.159577,12.139458,8783.0,0.157788,0
4,7.124785,310.472238,154.800236,3.269606,0.062448,3.632976,8693.0,0.201229,0


You can dump the first N frames using `s.dump(N)`.

## Link Trajectories, Streaming From and Updating the HDF5 File

In [8]:
with tp.PandasHDFStore('data.h5') as s:
    for linked in tp.link_df_iter(s, 3, neighbor_strategy='KDTree'):
        s.put(linked)

Frame 9: 563 trajectories present.


The original data is overwritten.

In [9]:
with tp.PandasHDFStore('data.h5') as s:
    frame_2_results = s.get(2)
    
frame_2_results.head()  # Display the first few rows.

,y,x,mass,size,ecc,signal,raw_mass,ep,frame,particle
0,5.644760,295.595642,249.545063,2.178398,0.182386,15.450905,8915.0,0.116651,2,2
1,6.359559,68.303687,227.102408,2.428999,0.082812,10.358149,9269.0,0.072415,2,7
2,6.438481,336.577209,209.061966,2.006389,0.113846,14.415090,8815.0,0.140979,2,3
3,6.902141,432.482819,346.652706,2.863077,0.485857,13.551911,9127.0,0.085406,2,5
4,8.217391,36.224806,256.105224,2.625996,0.144332,9.667605,9530.0,0.056592,2,6


## Framewise Data Interfaces

### Built-in interfaces

There are three different interfaces. You can use them interchangeably. They offer different performance advantages.

* [`PandasHDFStore`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L103-L161) -- fastest for a small (~100) number of frames
* [`PandasHDFStoreBig`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L164-L227) -- fastest for a medium or large number of frames
* [`PandasHDFStoreSingleNode`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L230-L320) -- optimizes HDF queries that access multiple frames (advanced)

### Writing your own interface

Trackpy implements a generic interface that could be used to store and retrieve particle tracking data in any file format. We hope that it can make it easier for researchers who use different file formats to exchange data. Any in-house format could be accessed using the same simple interface demonstrated above.

At present, the interface is implemented only for HDF5 files. To extend it to any format, write a class subclassing [`trackpy.FramewiseData`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L14-L86). This custom class must implement the methods `put`, `get`, `close`, and `__iter__` and the properties `max_frame` and `t_column`. Refer to the built-in classes in [framewise_data.py](https://github.com/soft-matter/trackpy/blob/master/trackpy/framewise_data.py) for examples to work from.